### Import Libraries

In [18]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from IPython.display import display
import json

### Parameters that will be passed in

In [ ]:
provider_id = None
provider_name = None
month_1_payments = None
month_2_payments = None
month_3_payments = None
month_4_payments = None

### Create a temporary SQLite database and populate it with our data

In [19]:
def setup_temp_database():
    conn = sqlite3.connect(':memory:')
    
    # Create provider payments table
    conn.execute('''
    CREATE TABLE provider_payments (
        month_number INTEGER,
        payment_amount DECIMAL(10,2),
        provider_id INTEGER,
        provider_name TEXT
    )
    ''')
    
    # Insert the payment data
    conn.execute('''
    INSERT INTO provider_payments (month_number, payment_amount, provider_id, provider_name)
    VALUES 
        (1, ?, ?, ?),
        (2, ?, ?, ?),
        (3, ?, ?, ?),
        (4, ?, ?, ?)
    ''', [month_1_payments, provider_id, provider_name,
         month_2_payments, provider_id, provider_name,
         month_3_payments, provider_id, provider_name,
         month_4_payments, provider_id, provider_name])
    
    return conn

### SQL Analysis Queries

In [20]:
PAYMENT_SUMMARY_QUERY = """
SELECT 
    provider_id,
    provider_name,
    SUM(payment_amount) as total_payments,
    AVG(payment_amount) as avg_monthly_payment,
    MAX(payment_amount) as highest_payment,
    MIN(payment_amount) as lowest_payment,
    (MAX(payment_amount) - MIN(payment_amount)) / MIN(payment_amount) * 100 as payment_fluctuation_pct
FROM provider_payments
GROUP BY provider_id, provider_name;
"""

MONTHLY_TREND_QUERY = """
SELECT 
    month_number,
    payment_amount,
    ROUND(((payment_amount / LAG(payment_amount, 1) OVER (ORDER BY month_number)) - 1) * 100, 2) as growth_rate
FROM provider_payments
ORDER BY month_number;
"""

PAYMENT_DISTRIBUTION_QUERY = """
SELECT
    provider_id,
    provider_name,
    COUNT(*) as num_months,
    SUM(CASE WHEN payment_amount > (SELECT AVG(payment_amount) FROM provider_payments) THEN 1 ELSE 0 END) as months_above_avg,
    SUM(CASE WHEN payment_amount < (SELECT AVG(payment_amount) FROM provider_payments) THEN 1 ELSE 0 END) as months_below_avg
FROM provider_payments
GROUP BY provider_id, provider_name;
"""

### Execute analysis

In [ ]:
def run_analysis(conn):
    # Get summary statistics
    summary_df = pd.read_sql_query(PAYMENT_SUMMARY_QUERY, conn)
    
    # Get monthly trends
    trends_df = pd.read_sql_query(MONTHLY_TREND_QUERY, conn)
    
    # Get distribution analysis
    distribution_df = pd.read_sql_query(PAYMENT_DISTRIBUTION_QUERY, conn)
    
    # Convert DataFrames to dictionaries for JSON serialization
    return {
        'summary': summary_df.to_dict('records'),
        'trends': trends_df.to_dict('records'),
        'distribution': distribution_df.to_dict('records')
    }

### Setup database

In [21]:
conn = setup_temp_database()

### Run analysis

In [22]:
results = run_analysis(conn)

### Display results

In [23]:
print("\nPayment Summary:")
display(pd.DataFrame(results['summary']))

print("\nMonthly Trends:")
display(pd.DataFrame(results['trends']))

print("\nPayment Distribution:")
display(pd.DataFrame(results['distribution']))


Payment Summary:


,provider_id,provider_name,total_payments,avg_monthly_payment,highest_payment,lowest_payment,payment_fluctuation_pct
0,None,None,None,None,None,None,None



Monthly Trends:


,month_number,payment_amount,growth_rate
0,1,None,None
1,2,None,None
2,3,None,None
3,4,None,None



Payment Distribution:


,provider_id,provider_name,num_months,months_above_avg,months_below_avg
0,None,None,4,0,0


### Close connection    

In [24]:
conn.close()

### Convert results to JSON for the notebook executor

In [25]:
json.dumps(results)

TypeError: Object of type DataFrame is not JSON serializable